In [1]:
# from eval import liHongEvaluator
from sklearn.neural_network import MLPRegressor
import random
import tensorflow as tf
# from neural_model import TFNeuralNetworkRegressor
# from get_onehotencoded_structure import get_structure1,dict_arm_to_structure1
import sys
import os
import numpy as np
from datetime import datetime
from math import log
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import random
import json
from keras.layers import LSTM
# from conversions import get_structure1,dict_arm_to_structure1
from sklearn.model_selection import train_test_split
import os
# from getData import campaign_def
import keras
#from getData import campaign_def
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import RepeatVector
from tflearn import DNN
from tflearn.layers.core import input_data, dropout, fully_connected 
from tflearn.layers.estimator import regression

import pickle

/home/anubha/anaconda3/envs/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/anubha/anaconda3/envs/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/anubha/anaconda3/envs/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/anubha/anaconda3/envs/venv/lib/python3.6/site-

Instructions for updating:
Colocations handled automatically by placer.


In [2]:
tf.reset_default_graph()

In [3]:
class liHongEvaluator():

    def __init__(self, main_folder, output_column_name, print_params = {}, use_non_stationarity = False, sampling_fraction=0.1, stochastic_param = [1,0]):
        self.main_folder = main_folder
        self.df_original = pd.read_csv(main_folder,engine='python')
#         self.df_original=self.df_original.drop(columns=['user_id','article_id'])
        print(self.df_original.head())

        all_columns = list(self.df_original.columns)
        self.output_column_names = all_columns[62:]
         

        self.input_columns = list(set(all_columns) - set(self.output_column_names))
        
        print(self.input_columns)
        self.arm_column_names = self.output_column_names
        self.logging_frequency = 1000
        self.number_features_in_context = len(self.input_columns)
        self.no_oflist_arms = len(self.output_column_names)
        total_arms =40

        #For testing, Comment out
        self.count_not_found = 0
        self.found_in_exclusion = 0
        self.found_in_set = 0
        self.flag = True
        x = self.df_original[self.input_columns].values
        x_scaled=pd.DataFrame(x,columns = self.input_columns)
        self.unique = self.df_original[self.output_column_names].drop_duplicates()
        self.unique = self.unique.reset_index(drop=True)

        self.y=self.df_original[self.output_column_names].values
        #To simulate stochasticity set this paramater
        self.stochastic_param = stochastic_param
        x_tr, x_test , y_tr, y_test = train_test_split(x_scaled,self.y,test_size=0.3,shuffle=False)
        x_train ,x_valid,y_train, y_valid = train_test_split(x_tr,y_tr,test_size=0.3,shuffle=False)
 
        self.x_training = x_train.values.tolist() #list(map(float,x_train.values))#pd.DataFrame(x_tr, columns = self.input_columns)
        self.y_training = y_train #pd.DataFrame(y_tr, columns = [self.output_column_names])
        self.x_valid=x_valid.values.tolist()
        self.y_valid=y_valid
        self.x_testing =x_test.values.tolist()#pd.DataFrame(x_test, columns = self.input_columns)
        self.y_testing = y_test# pd.DataFrame(y_test, columns = [self.output_column_names])
        x_train_df = pd.DataFrame(x_tr, columns = self.input_columns)
        x_test_df = pd.DataFrame(x_test, columns = self.input_columns)
        
        self.e=0.2
        self.decay=0.998
        
        print ('x_traing',self.x_training[0])
        print ('x_valid',self.x_valid[0])
        print ('x_test' ,self.x_testing[0])


        random.seed(9001)

        self.non_stationarity_steps = 25000
        
    def get_size_input(self):
        return self.df.shape[0]

    def get_no_oflist_arms(self):
        return self.no_oflist_arms

    def get_dataset_shape(self):
        return self.df.shape

    def get_number_features_in_context(self):
        return self.number_features_in_context

    #rename to get_one_hot_structure_array
    def get_structure(self):
#         print(np.unique(self.y))
        structure=self.unique
        #print 'structure', structure
        return structure
    #rename to get_one_hot_structure
    def dict_arm_to_structure(self):
        dict_str ={}
        y_unique = self.get_structure()
        print (y_unique.head())
        for i ,r in y_unique.iterrows():
            dict_str[i] = r.values.tolist()
        return dict_str

    def get_exclusions(self):
        arms_set_dict ,self.exclusion_dict = campaign_def(self.main_folder)
        return self.exclusion_dict


    def evaluate(self,policy):
        
        dict_arm_structure = self.dict_arm_to_structure()
        score=0
        total_arms=40
        start_time = datetime.now()
        q=0
        res_rew=[]
        rew_est=0
        count = 0
        

        
        print("now training model")
        for state, right_arm in zip(self.x_training,self.y_training):
            
            q+=1
            if(q%100 == 0):
                print('Hey there ', q)
                res_rew.append(rew_est)
                rew_est=0
            
            
            import random
            rand = random.uniform(0, 1)
            
            action_indices = []
            if(rand < self.e):
                self.e=self.e*self.decay
                selected_action =[]
                
                while len(selected_action) <1:
                    random_index = np.random.randint(0,total_arms)
                    temp = self.unique.iloc[random_index].values.tolist()
                    if(temp not in selected_action):
                        selected_action.append(temp)
                        action_indices.append(random_index)
            else:        
            
                action_indices, selected_action = policy.get_action(state)
            count=count+1
            
            if count % self.logging_frequency == 0 :
                print('training set evaluated %d' % count)
            reward=0

            for action_index, arm in zip(action_indices, selected_action) :
                reward=0
                if arm == right_arm.tolist():

                    reward = 1
                    rew_est+=1

                state2 =list(state) + (arm)
                policy.update_model(reward,state2,arm,True, action_index)

            
            #print("!!!", right_arm)
#             state1 = list(state) + list(right_arm)
#             policy.update_model(1,state1,right_arm,True)
            
#             import random
#             random_sample = list(self.unique.iloc[random.randint(0,39)])
#             if random_sample!=list(right_arm):
#                 policy.update_model(0,state + random_sample,random_sample, True)

#             pairs = list(zip(self.x_training,self.y_training))  # make pairs out of the two lists
#             pairs = random.sample(pairs, 1)  # pick 3 random pairs
#             A1, B1 = zip(*pairs)  # separate the pairs
#             for k in range(1):
#                 if(list(B1[k])!=list(right_arm)):
#                     state1 = list(A1[k]) + list(B1[k])
#                     policy.update_model(0,state1,B1[k],True)
                    
            
#             for index, row in self.unique.iterrows():
#                 row = row.to_numpy()
#                 state2 =list( state) + list(row)
#                 #print(state)
#                 if(list(row)==list(right_arm)):
#                     policy.update_model(1,state2,right_arm,True)
#                 else:
#                     policy.update_model(0,state2,row,True)
            
# # # #             print('state',state)
            
# # #         policy.train_model(self.x_training, self.y_training)
        print("model trained")
        
        count = 0
        for state, right_arm in zip(self.x_valid, self.y_valid):
            
            q+=1
            if(q%100 == 0):
                res_rew.append(rew_est)
                rew_est=0

            import random
            rand = random.uniform(0, 1)

            action_indices = []
            if(rand < self.e):
                self.e=self.e*self.decay
                selected_action =[]
                
                while len(selected_action) <1:
                    random_index = np.random.randint(0,total_arms)
                    temp = self.unique.iloc[random_index].values.tolist()
                    if(temp not in selected_action):
                        selected_action.append(temp)
                        action_indices.append(random_index)
            else:        
            
                action_indices, selected_action = policy.get_action(state)
            
            count=count+1
            if count % self.logging_frequency == 0 :
                print('validation set evaluated %d' % count)
            reward=0


            for action_index, arm in zip(action_indices, selected_action) :
                reward=0
                if arm == right_arm.tolist():

                    reward = 1
                    rew_est+=1

                state2 =list(state) + (arm)
                policy.update_model(reward,state2,arm,True, action_index)

        
        
##############################################TESTING##############################################
        
        
        count =0
        
        precision=0
        dict_recall= {}
        dict_impressions = {}
        dict_clicks = {}
        
        for k in self.unique.values:
            dict_impressions[tuple(k)] =0
            dict_clicks[tuple(k)]=0
        
        ndgc=0
        
        for state, right_arm in zip(self.x_testing, self.y_testing):
                                    
            import random
            rand = random.uniform(0, 1)

            action_indices = []
            if(rand < self.e):
                self.e=self.e*self.decay
                selected_action =[]
                
                while len(selected_action) <1:
                    random_index = np.random.randint(0,total_arms)
                    temp = self.unique.iloc[random_index].values.tolist()
                    if(temp not in selected_action):
                        selected_action.append(temp)
                        action_indices.append(random_index)
            else:        
            
                action_indices, selected_action = policy.get_action(state)
                
            q+=1
            if(q%100 == 0):
                res_rew.append(rew_est)
                rew_est=0
            


            y_value = right_arm
    
            

            count=count+1
            self.stime = datetime.now()
            if count % self.logging_frequency == 0 :
                print('testing set evaluated %d' % count)
# #             print('initial', len(state))
            
            match_found = False
            ps =0
            ps_sum=0
            i=0
            num_ndgc=0
            d_ndgc=0
            k_ndgc=0
            for action_index, arm in zip(action_indices, selected_action) :
                    reward=0
                    dict_impressions[tuple(arm)]+=1

                    if arm == right_arm.tolist():
                        rew_est +=1

                        match_found = True
                        reward = 1
                        score = score+1

                        ps+=1
                        ps_sum += ps/(i+1)
                        i=i+1
                        
                        if(i+1==1):
                            d_ndgc+=1
                        else:
                            d_ndgc+=1/(log(i+1))
                        num_ndgc = num_ndgc+1
                        
                        dict_clicks[tuple(y_value)] +=1


                    state2 =list(state) + (arm)
                    policy.update_model(reward,state2,arm,True,action_index)
                    
            if(match_found):
                precision =precision + ps_sum
                
            for i in range(num_ndgc):
                if(i+1 == 1):
                    k_ndgc+=1
                else:
                    k_ndgc+= 1/(log(i+1))
            
            if(k_ndgc!=0):
                ndgc = ndgc + (d_ndgc/k_ndgc)
            


        print('accuracy %f' % ( float(score) / count ))
        
        ctr = 0 
        for key, value in dict_impressions.items():
            if(value!=0):
                ctr += dict_clicks[key]/value
        print('ctr',ctr/len(dict_impressions.keys()))
        
        with open('result_reward_single_news_ts1.pickle', 'wb') as handle:
            pickle.dump(res_rew, handle)
        
        print ("precision")
        print (precision/len(self.x_testing))
        
                
        print ("NDGC")
        print (ndgc/len(self.x_testing))
        




In [4]:
pwd

'/media/anubha/Seagate Expansion Drive/china'

In [5]:
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import Model
data_path = '/media/anubha/Seagate Expansion Drive/china'
class TFNeuralNetworkRegressor():
    
    


    def __init__(self, number_features_in_context, number_features_in_structure):
        
        
        self.pre_model = Sequential()
        self.pre_model.add(LSTM(32, batch_input_shape=(4,1,number_features_in_context+number_features_in_structure),stateful=True,name='layer1', return_sequences=True))
        #self.pre_model.add(RepeatVector(1))
        self.pre_model.add(LSTM((number_features_in_context+number_features_in_structure), return_sequences=True, stateful = True)) 
        self.pre_model.add(Dense((number_features_in_context+number_features_in_structure),activation ='sigmoid'))
        self.pre_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#         checkpointer = ModelCheckpoint(filepath=data_path + '/model-{epoch:02d}.hdf5', verbose=0)
        print(self.pre_model.summary())
    
        self.model = Sequential()
        #First Hidden Layer
#         self.model.add(LSTM(120, batch_input_shape=(100,number_features_in_context+number_features_in_structure,1), stateful=True))

        self.model.add(Dense(60, activation='tanh', kernel_initializer='random_normal', input_shape=(32,)))
        
        
#         self.model.add(Dense(80, activation='tanh', kernel_initializer='random_normal'))
        
        self.model.add(Dense(30, activation='tanh', kernel_initializer='random_normal'))
        

        self.model.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
        # For a binary classification problem
        
        self.model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
        print(self.model.summary())
    
    
        layer_name = 'layer1'
        self.intermediate_layer_model = Model(inputs=self.pre_model.input,outputs=self.pre_model.get_layer(layer_name).output)
        
        self.intermediate_batch_output_history = None
       
    def fit(self, X, Y):
        X = X.reshape(X.shape[0],1,X.shape[1])
#         intermediate_output = self.intermediate_layer_model.predict(data)
        self.pre_model.fit(X,X,batch_size=4, verbose=0)
        intermediate_output = self.intermediate_layer_model.predict(X,batch_size=4)
        intermediate_output = intermediate_output.reshape((intermediate_output.shape[0], np.prod(intermediate_output.shape[1:])))
        
        self.model.fit(intermediate_output, Y, epochs=1, verbose=0)
        
        
   
    def partial_fit(self, X,Y, action_index):
        
        if self.intermediate_batch_output_history is None:
            
            aux = np.concatenate([X]*40,axis = 0)
            aux = aux.reshape(aux.shape[0], 1,aux.shape[1])
            intermediate_output = self.intermediate_layer_model.predict(aux,batch_size=4)
            intermediate_output = intermediate_output.reshape((intermediate_output.shape[0], np.prod(intermediate_output.shape[1:])))
            self.intermediate_batch_output_history = intermediate_output
            print('intermediate output shape',intermediate_output.shape)
        self.model.fit(np.array([self.intermediate_batch_output_history[action_index]]), Y, epochs=10, verbose=0)
   
    def predict(self, X):
        #print('input shape', X.shape)
        X = X.reshape(X.shape[0], 1,X.shape[1])
        intermediate_output = self.intermediate_layer_model.predict(X,batch_size=4)
        
#       print('output shape', intermediate_output.shape)
#         intermediate_from_pre_model = self.pre_model.get_layer('lstm_22').output
        intermediate_output = intermediate_output.reshape((intermediate_output.shape[0], np.prod(intermediate_output.shape[1:])))
        self.intermediate_batch_output_history = intermediate_output
        return self.model.predict(intermediate_output)

In [6]:
def dict_arm_to_structure1():
    with open('total_comb_structured_dict') as f:
        data = json.load(f)
        data1={}
        for i in data.keys():
            j=str(i)
            data1[j]=data[i]
        return data1

def get_structure1():
	total_comb_structured_dict=dict_arm_to_structure1()
	total_structure_array= np.asarray(list(total_comb_structured_dict.values()))
	return total_structure_array

def num_to_key():
    with open('dict_structure_to_arm') as f:
        data = json.load(f)
        data1={}
        for i in data.keys():

            j=str(i)
            data1[j]=data[i]
        return data1


In [7]:
#150 -50-0.7 [150 - 100-50]
class single_model_policy():
    #Taking batch sizes of 1000
    def __init__(self, number_features_in_context, structure, epsilon, training_dropout, prediction_dropout,batch_size = 100):
        self.last_action = None
        self.last_reward = None
        self.last_state = None
        self.num_arms = structure.shape[0]
        self.structure_features = structure.shape[1]
        self.epsilon = epsilon
        self.experience = {}
        self.i=0
        self.total_features = number_features_in_context + self.structure_features
        self.train_batch_size=120
        self.mini_batch=40
        dict_str ={}
        self.cnt=0
      
        for i ,r in structure.iterrows():
            dict_str[i] = r.values.tolist()
            
        
     
    
        self.structure = dict_str
#         print (self.structure)
#         self.batch_size = batch_size
        self.last_s=[]
        self.last_r=[]
        #if self.batch_size:
        self.iter = 0
        self.dict_update_hist = {}
#         print(self.structure)
        # self.model=TFNeuralNetworkRegressor(self.total_features, training_dropout, prediction_dropout)
        self.model=TFNeuralNetworkRegressor(number_features_in_context, self.structure_features)
        


    #Choose best action for given context
    def get_action(self, context):
        #action should be category ID
        best_action = self.get_best_arm(context)
        arms=[]
        for i in best_action:
            arms.append(self.structure[i])
            
        return best_action, arms

    #Loop over the structures of every arm for a given context and return the best prediction
    def get_best_arm(self,context):

        best_score = 0
        best_arm = None

        inputs = []
        arms = []
        for arm,structure in self.structure.items():
#             print((structure))
#             print (list())
            state = list(context) + list(structure)
#             print(len(state))
            inputs.append(state)
            arms.append(arm)

        scores = [ elt[0] for elt in list(self.model.predict(np.array(inputs))) ]
        #print(scores)
        n=0
        select_arm=[]
        s = sorted(range(len(scores)),reverse = True,  key=lambda k: scores[k])

        return s[:1]

    
    def train_model(self, x_train, y_train):
        self.model.fit(x_train,y_train)
    #Update model

    def update_model(self, reward,last_state=[] ,last_action=[],update_in_train=False, action_index = 0):
        #Immediate Updates
        if(len(last_action) != 0):
            self.last_action = last_action
        if(len(last_state) !=0):
            self.last_state = last_state
        self.last_reward = reward
        self.model.partial_fit(np.array([self.last_state]), np.array([[self.last_reward]]), action_index)
#         print (batch_size)
        if update_in_train:
            if(len(self.last_s)!=120):
#             if(self.iter == 0 or self.iter%self.train_batch_size != 0):
#                 if(reward==0):
#                     self.cnt+=1
                    
#                 if(self.cnt<int(0.5*self.train_batch_size) or reward==1):

#                     if(self.last_state not in self.last_s):
                self.last_s.append(self.last_state)
                self.last_r.append(self.last_reward)

            else:

                for iteration in range(1,10):
                    if(iteration%500==0):
                        print("Iteration : %d" % iteration)
                    num_to_sample = min(len(self.last_s), self.mini_batch)
                    random_indices=random.sample(range(0,len(self.last_s)),num_to_sample)
                    x_train = []
                    y_train = []
                    for index in random_indices:

                        x_train.append(self.last_s[index])
                        y_train.append([self.last_r[index]])

                    self.model.fit(np.array(x_train),np.array(y_train))
                self.last_s=[]
                self.last_r=[]
                self.cnt=0
            self.iter+=1


In [8]:
training_dropout, prediction_dropout =  0.5, 0.5
epsilon = 0.00001

main_folder = 'china_structure27.1.20.csv'

target_evaluator = liHongEvaluator(main_folder, ['article_id'] , {}, False, 0.1, [1,0])

print("Created Evaluator")


   frequency_rating    counts  activeness    rating  timestamp  longitude  \
0          0.174419  0.016938     1.00000  0.872302   0.000000   0.904337   
1          0.174419  0.163279     1.00000  0.872302   0.000099   0.927004   
2          0.906977  0.163279     0.00000  0.140288   0.000099   0.927004   
3          1.000000  0.163279     0.96281  0.968562   0.000099   0.927004   
4          0.992248  0.163279     1.00000  0.998801   0.000099   0.927004   

   latitude  is_installed       age  phone_brand_0  ...  6.66657E+018  \
0  0.573701             1  0.177419              0  ...             0   
1  0.446858             1  0.161290              0  ...             0   
2  0.446858             1  0.161290              0  ...             0   
3  0.446858             1  0.161290              0  ...             0   
4  0.446858             1  0.161290              0  ...             0   

   7.16711E+018  7.31625E+018  7.34834E+018  7.35097E+018  7.46008E+018  \
0             0        

In [9]:
df=pd.read_csv(main_folder)

In [10]:
structure= target_evaluator.get_structure()
number_features_in_context = target_evaluator.get_number_features_in_context()

In [11]:
smp = single_model_policy(number_features_in_context, structure, epsilon, training_dropout, prediction_dropout)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (LSTM)                (4, 1, 32)                25856     
_________________________________________________________________
lstm_1 (LSTM)                (4, 1, 169)               136552    
_________________________________________________________________
dense_1 (Dense)              (4, 1, 169)               28730     
Total params: 191,138
Trainable params: 191,138
Non-trainable params: 0
_________________________________________________________________
None
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 60)                1980      
_________________________________________________________________
dense_3 (Dense)              (None, 30)                1830      
_________________________________________________________________

In [12]:
target_evaluator.evaluate(smp)


    like groupon  1 free  1 reputation  And the Church  Bank financing  \
0              0       0             0               0               0   
1              0       0             0               1               0   
2              0       0             0               0               0   
3              0       0             0               0               0   
4              0       0             0               0               0   

   Buy class  Consumer Finance  Cozy 1  Custom label  Customized 1  ...  \
0          0                 0       0             0             0  ...   
1          0                 0       0             1             0  ...   
2          0                 0       0             0             0  ...   
3          0                 0       0             0             0  ...   
4          0                 0       0             0             0  ...   

   6.66657E+018  7.16711E+018  7.31625E+018  7.34834E+018  7.35097E+018  \
0             0             0

Hey there  31200
Hey there  31300
Hey there  31400
Hey there  31500
Hey there  31600
Hey there  31700
Hey there  31800
Hey there  31900
Hey there  32000
training set evaluated 32000
Hey there  32100
Hey there  32200
Hey there  32300
Hey there  32400
Hey there  32500
Hey there  32600
Hey there  32700
Hey there  32800
Hey there  32900
Hey there  33000
training set evaluated 33000
Hey there  33100
Hey there  33200
Hey there  33300
Hey there  33400
Hey there  33500
Hey there  33600
Hey there  33700
Hey there  33800
Hey there  33900
Hey there  34000
training set evaluated 34000
Hey there  34100
Hey there  34200
Hey there  34300
Hey there  34400
Hey there  34500
Hey there  34600
Hey there  34700
Hey there  34800
Hey there  34900
Hey there  35000
training set evaluated 35000
Hey there  35100
Hey there  35200
Hey there  35300
Hey there  35400
model trained
validation set evaluated 1000
validation set evaluated 2000
validation set evaluated 3000
validation set evaluated 4000
validation set eval

In [13]:
batch size 1000
iterations :10 
    _____________
accuracy 0.250963
ctr 0.04623804173875244
precision
0.25096348791758755
NDGC
0.362063779462863

SyntaxError: invalid syntax (<ipython-input-13-d2b7a3f08c64>, line 1)

In [ ]:
batch size : 500
iterations : 50
    
    ____________________
    
accuracy 0.410747
ctr 0.14885359288583871
precision
0.4107472282598769
NDGC
0.5925829892693465

In [ ]:
accuracy 0.583919
ctr 0.20346427770087075
precision
0.5839193409925295
NDGC
0.8424175375289795

3 layer
batch size=100
iterations :50

In [ ]:

3 layer 120 -80-40
batch size : 100
iter = 50


accuracy 0.582385
ctr 0.22105107629248555
precision
0.5823850663939067
NDGC
0.8402040471741848

In [ ]:
accuracy 0.589709
ctr 0.20182517504152558
precision
0.5897094010849513
NDGC
0.8507708285107646

In [ ]:
LSTM
batch size 80
encoding to 32

accuracy 0.505872
ctr 0.15674250904718953
precision
0.5058722533744908
NDGC
0.729819391266622

In [ ]:
LSTM
Batch size 120
encoding 16

accuracy 0.541014
ctr 0.18200622240655773
precision
0.5410144477524703
NDGC
0.7805188608216831

In [ ]:
LSTM
Batch size 120
encoding 16
simultaneous updates

accuracy 0.570622
ctr 0.18662399152101622
precision
0.5706222944711319
NDGC
0.8232339544540916

In [ ]:
LSTM
Batch size 240
encoding 16
simultaneous updates

accuracy 0.648834
ctr 0.1964724422136122
precision
0.6488337686533088
NDGC
0.93606926039732